In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from tensorflow.keras import applications
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import optimizers
from tensorflow.keras import Sequential, Model
from tensorflow.keras.layers import Dropout, Flatten, Dense
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras import backend as k
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.callbacks import TensorBoard, LearningRateScheduler

VGG16: image(224,224)

In [ ]:
img_width, img_height = 224,224
train_data_dir = "images_translearn/training_set"
validation_data_dir = "images_translearn/testing_set"

In [ ]:
model = applications.VGG16(weights='imagenet', include_top=False, 
                           input_shape= (img_width, img_height,3))

In [ ]:
model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

New dataset is similar but small compared to original's.

In [ ]:
# freeze all the VGG16 layers and train only the classifier
for layer in model.layers:
  layer.trainable = False

In [ ]:
#adding custom layers -ANN
x = model.output
x = Flatten()(x)
x = Dense(1024, activation="relu")(x)
x = Dropout(0.5)(x)
x = Dense(1024, activation="relu")(x)
predictions = Dense(2, activation="softmax")(x)

In [ ]:
model_final = Model(inputs = model.input, outputs = predictions)

In [ ]:
#compile
model_final.compile(loss="categorical_crossentropy",
                    optimizer = optimizers.SGD(lr=0.0001, momentum=0.9),
                    metrics=["accuracy"])

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   horizontal_flip=True,
                                   fill_mode = "nearest",
                                   zoom_range=0.3,
                                   width_shift_range=0.3,
                                   height_shift_range=0.3,
                                   rotation_range=30)

test_datagen = ImageDataGenerator(rescale=1./255,
                                   #horizontal_flip=True,
                                   #fill_mode = "nearest",
                                   #zoom_range=0.3,
                                   #width_shift_range=0.3,
                                   #height_shift_range=0.3,
                                   #rotation_range=30)
)

In [ ]:
train_generator = train_datagen.flow_from_directory(train_data_dir,
                                                    target_size=(img_height,
                                                                 img_width),
                                                    batch_size = 32,
                                                    class_mode = "categorical")

validation_generator = test_datagen.flow_from_directory(validation_data_dir,
                                                    target_size=(img_height,
                                                                 img_width),
                                                    class_mode = "categorical")

Found 316 images belonging to 2 classes.
Found 126 images belonging to 2 classes.


In [ ]:
checkpoint = ModelCheckpoint("vgg16_fire.h5",
                             monitor="val_loss",
                             verbose=1,
                             save_best_only=True,
                             save_weights_only=False,
                             mode="auto",
                             save_freq=1)

early = EarlyStopping(monitor='val_loss',
                      min_delta=0,
                      patience=10,
                      verbose=1,
                      mode="auto")

In [ ]:
import datetime

In [ ]:
t0 = datetime.datetime.now()
print(t0)

2020-11-29 13:04:51.695555


In [ ]:
model_final.fit(train_generator,
                steps_per_epoch = len(train_generator),
                epochs = 100,
                validation_data=validation_generator,
                validation_steps= len(validation_generator),
                verbose = 1,
                callbacks = [checkpoint, early])

Epoch 1/100
10/10 [==============================] - 105s 10s/step - loss: 0.7220 - accuracy: 0.5443 - val_loss: 0.7203 - val_accuracy: 0.4683
Epoch 2/100
10/10 [==============================] - 116s 12s/step - loss: 0.6965 - accuracy: 0.5854 - val_loss: 0.7680 - val_accuracy: 0.3810
Epoch 3/100
10/10 [==============================] - 109s 11s/step - loss: 0.6838 - accuracy: 0.5886 - val_loss: 0.7994 - val_accuracy: 0.2619
Epoch 4/100
10/10 [==============================] - 109s 11s/step - loss: 0.6794 - accuracy: 0.5949 - val_loss: 0.8488 - val_accuracy: 0.3095
Epoch 5/100
10/10 [==============================] - 105s 10s/step - loss: 0.6413 - accuracy: 0.6297 - val_loss: 0.8721 - val_accuracy: 0.2460
Epoch 6/100
10/10 [==============================] - 105s 10s/step - loss: 0.6454 - accuracy: 0.6139 - val_loss: 0.9525 - val_accuracy: 0.3095
Epoch 7/100
10/10 [==============================] - 106s 11s/step - loss: 0.6004 - accuracy: 0.6962 - val_loss: 0.9554 - val_accuracy: 0.2381

In [ ]:
t1 = datetime.datetime.now()
print(t1-t0)

0:21:42.613204


In [ ]:
from tensorflow.keras.models import load_model

model_final.save("my_vgg16_fire.h5")
print("save!")

save!


### Compare execution time and accuracy of two models (Question 3 vs. Question 4)

* CNN model (full dataset 4122 images)  : time: 29min12s       accuracy: 90%
* vg16 model (small dataset 442 images) : time: 21min43s       accuracy: 70%

The differences in accuracy between the 2 models are because the dataset for Q4 is much smaller compared to the originals in Q3.